In [422]:
# Necessary packages
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np, pandas as pd

In [423]:
orig_data_dir = "../../data/processed_orig_data/"
gen_data_dir = "../../data/generated_data/"

# Load data

In [424]:
dataset = 'stocks'           # sine, stocks, air, energy
perc = 100
model = 'vae_conv_I'

# orig data

sample_file_name = orig_data_dir + f'{dataset}_subsampled_train_perc_{perc}.npz'
loaded = np.load(sample_file_name)
orig_data = loaded['data']
print(orig_data.shape)


# gen data
sample_file_name = gen_data_dir + f'{model}_gen_samples_{dataset}_perc_{perc}.npz'
loaded = np.load(sample_file_name)
gen_data = loaded['data']
print(gen_data.shape)



(3661, 24, 6)
(3661, 24, 6)


In [425]:
N, T, D = orig_data.shape

# Scale Data

In [426]:
class MinMaxScaler():
    """Min Max normalizer.
    Args:
    - data: original data

    Returns:
    - norm_data: normalized data
    """
    def fit(self, data):    
        self.mini = np.min(data, 0)
        numerator = data - self.mini
        self.range = np.max(data, 0) - self.mini
        norm_data = numerator / (self.range + 1e-7)
        return norm_data
    
    def inverse_transform(self, data):
        data *= self.range
        data += self.mini
        return data

In [427]:
scaler_orig = MinMaxScaler( )  
scaled_orig_data = scaler_orig.fit(orig_data)
print('scaled orig data shape: ', scaled_orig_data.shape)
print('scaled orig data mean: ', scaled_orig_data.mean())

scaler_gen = MinMaxScaler( )  
scaled_gen_data = scaler_gen.fit(gen_data)
print('scaled gen data shape: ', scaled_gen_data.shape)
print('scaled gen data mean: ', scaled_gen_data.mean())

scaled orig data shape:  (3661, 24, 6)
scaled orig data mean:  0.2892493093648157
scaled gen data shape:  (3661, 24, 6)
scaled gen data mean:  0.22435862


In [428]:
print(scaled_orig_data.mean(axis=0).mean(axis=1))

print(scaled_gen_data.mean(axis=0).mean(axis=1))

[0.28818122 0.28822461 0.28841546 0.2886137  0.28881881 0.28902849
 0.28923789 0.28944822 0.28965798 0.28987007 0.29008394 0.29020806
 0.2902559  0.29032908 0.29047756 0.29022314 0.28967846 0.28930371
 0.28906335 0.28887134 0.28849906 0.28831381 0.28850287 0.28867671]
[0.22716229 0.22807677 0.22378354 0.22308762 0.21880533 0.2243675
 0.2224319  0.22161019 0.22354358 0.22220726 0.22429948 0.22465378
 0.2200628  0.22703253 0.2212137  0.22018711 0.22454269 0.22465235
 0.22623338 0.22607864 0.22926873 0.22505854 0.23090355 0.22534479]


In [429]:
print('scaled orig data min/max: ', scaled_orig_data.min(), scaled_orig_data.max())
print('scaled orig scaled_gen_data min/max: ', scaled_gen_data.min(), scaled_gen_data.max())

scaled orig data min/max:  0.0 0.9999999999999988
scaled orig scaled_gen_data min/max:  0.0 0.99999994


In [430]:
final_orig = scaled_orig_data.mean(axis=2)
final_gen = scaled_gen_data.mean(axis=2)
print('final_orig shape: ', final_orig.shape)
print('final_gen shape: ', final_gen.shape)

final_orig shape:  (3661, 24)
final_gen shape:  (3661, 24)


In [431]:
final_tsne_input = np.concatenate((final_orig, final_gen), axis = 0)
# final_tsne_input = final_orig

print('final_tsne_input shape: ', final_tsne_input.shape)

final_tsne_input shape:  (7322, 24)


In [434]:
# TSNE analysis
tsne = TSNE(n_components = 2, perplexity = 40, n_iter = 300)
tsne_results = tsne.fit_transform(final_tsne_input)


colors = ["red" for i in range(N)] + ["blue" for i in range(N)] 

# Plotting
f, ax = plt.subplots(1)

plt.scatter(tsne_results[:N,0], tsne_results[:N,1], 
            c = colors[:N], alpha = 0.2, label = "Original")
plt.scatter(tsne_results[N:,0], tsne_results[N:,1], 
            c = colors[N:], alpha = 0.2, label = "Synthetic")

ax.legend()

plt.title('t-SNE plot')
plt.xlabel('x-tsne')
plt.ylabel('y_tsne')
plt.savefig("tsne-results.png", dpi=300)  
plt.show()